In [7]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser

def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [8]:
headers = {
    'User-Agent': 'GOLUB CAPITAL BDC, Inc.'
}
filing_links = pd.read_excel(
    "../GBDC__sec_filing_links.xlsx")
filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,Act,Film number,Accession number,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2013-05-03,2013-03-31,34,13810352,0001144204-13-026113,https://www.sec.gov/Archives/edgar/data/147676...
1,10-Q,Quarterly report [Sections 13 or 15(d)],2013-08-08,2013-06-30,34,131020072,0001144204-13-043799,https://www.sec.gov/Archives/edgar/data/147676...
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2013-12-03,2013-09-30,34,131254591,0001144204-13-065322,https://www.sec.gov/Archives/edgar/data/000147...
3,10-Q,Quarterly report [Sections 13 or 15(d)],2014-02-06,2013-12-31,34,14578102,0001144204-14-006255,https://www.sec.gov/Archives/edgar/data/000147...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2014-05-08,2014-03-31,34,14823110,0001144204-14-028416,https://www.sec.gov/Archives/edgar/data/000147...


In [9]:
start_row = 30
end_row = 42   

for index, row in filing_links.iloc[start_row:end_row].iterrows():
    print(row['Reporting date'],row['Filings URL'])
    # webbrowser.open_new_tab(row['Filings URL'])
    
    

2020-09-30 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676520000195/gbdcfy202010-k.htm
2020-12-31 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676521000023/gbdcfy202110-qq1.htm
2021-03-31 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676521000073/gbdcfy202110-qq2.htm
2021-06-30 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676521000153/gbdcfy202110-qq3.htm
2021-09-30 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676521000176/gbdcfy202110-k.htm
2021-12-31 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676522000020/gbdcfy202210-qq1.htm
2022-03-31 00:00:00 https://www.sec.gov/Archives/edgar/data/0001476765/000147676522000049/gbdcfy202210-qq2.htm
2022-06-30 00:00:00 https://www.sec.gov/Archives/edgar/data/1476765/000147676522000078/gbdc-20220630.htm
2022-09-30 00:00:00 https://www.sec.gov/Archives/edgar/data/1476765/000147676522000103/gbdc-20220930.htm
2022-12-31 00:00: